In [1]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [2]:
ngl_base = 'https://neuromancer-seung-import.appspot.com/' # this will point to the public bucket


In [3]:
def make_json(name, seg_ids):
    state = {
        "layers": [
            {
            "source": "graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod",
            "type": "segmentation_with_graph",
            "selectedAlpha": 0.3,
            "segments": seg_ids,
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "graphOperationMarker": [
                {
                "annotations": [],
                "tags": []
                },
                {
                "annotations": [],
                "tags": []
                }
            ],
            "pathFinder": {
                "color": "#ffff00",
                "pathObject": {
                "annotationPath": {
                    "annotations": [],
                    "tags": []
                },
                "hasPath": False
                }
            },
            "name": name
            },
            {
            "type": "segmentation",
            "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
            "objectAlpha": 0.1,
            "hideSegmentZero": False,
            "ignoreSegmentInteractions": True,
            "segmentColors": {
                "1": "#bfbfbf",
                "2": "#d343d6"
            },
            "segments": [
                "1",
                "2"
            ],
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "name": "neuropil"
            }
        ],
        "navigation": {
            "pose": {
            "position": {
                "voxelSize": [
                4.300000190734863,
                4.300000190734863,
                45
                ],
                "voxelCoordinates": [
                38930,
                138652,
                388
                ]
            }
            },
            "zoomFactor": 2
        },
        "showAxisLines": False,
        "showDefaultAnnotations": False,
        "perspectiveOrientation": [
            0,
            1,
            0,
            0
        ],
        "perspectiveZoom": 7704.152500126457,
        "showSlices": False,
        "selectedLayer": {
            "layer": "neuropil"
        },
        "layout": {
            "type": "3d",
            "orthographicProjection": True
        }
        }

    return state

In [114]:
# import dataframe
leg_df = pd.read_pickle('pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
wing_df = pd.read_pickle('preMN_to_MN_wing_v604.pkl')

In [115]:
print(leg_df.shape)
leg_df = leg_df[leg_df.index.get_level_values('cell_type') != 'Gial_error']
print(leg_df.shape)

(1460, 69)
(1460, 69)


In [116]:
leg_df.to_pickle('pre_to_mn_df_matched_typed_with_nt_v604_20230526.pkl')

# Preferred pool for different classes neurons

In [87]:
# Preferred pool for intersegmental neurons
# generate a link based on a preferred_pool
# local and intersegmental preMNs
All = slice(None)
leg_df = leg_df.loc[leg_df.index]

cell_class = leg_df.index.get_level_values('cell_class').unique()
idx = pd.IndexSlice
for cc in cell_class:

    class_df = leg_df.loc[idx[cc]]
    pool_keys = class_df.index.get_level_values('preferred_pool').unique()

    for pl in pool_keys:
        seg_ids = class_df.loc[pl,:].index.get_level_values('segID')
        formatted_ids = [str(x) for x in seg_ids]
        
        state = make_json(name = pl,
                    seg_ids = formatted_ids)
        new_id = client.state.upload_state_json(state)
        client.state.build_neuroglancer_url(new_id, ngl_base)

        # save json 
        with open('leg_jsons/{}_{}.json'.format(cc,pl), "w") as outfile:
            json.dump(state, outfile)

# Make Jsons for hemilineages

In [119]:
leg_df = pd.read_pickle('pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
cell_type_dict = {
    'RVD':'8A',
    '8A_':'8A',
    '16B_':'16B',
    '16B_small':'16B',
    'Rcore_':'8A',
    '24B_':'15B',
    '8A-':'8A',
    '22A_':'22A',
    '8A? 16B?':'16A',
    '21A_':'19A',
    '9Ac_':'9A',
    '13B_':'13B',
    '1B_':'1B',
    '22A?':'22A',
    'R0':'15B',
    'R1':'8A',
    'R2':'8A',
    'R3':'8A',
    'R4':'8A',
    'R5':'8A',
    'R6':'8A',
    '4B_':'4B',
    '4B-':'4B',
    '4B+':'4B',
    '14A_':'14A',
    'RCore':'8A',
    '8A- Rind':'8A',    
    '8A+':'8A',
}
cell_types = leg_df.index.get_level_values('cell_type').unique().to_numpy()
leg_idx = leg_df.index.to_frame().set_index('cell_type',drop=False)

for ct in cell_type_dict.keys():
    leg_idx.loc[ct,'cell_type'] = cell_type_dict[ct]

cell_types_ = leg_idx['cell_type'].unique()

leg_df.index = pd.MultiIndex.from_frame(leg_idx)
class_df = leg_df.loc['local']
cell_types = class_df.index.get_level_values('cell_type').unique().to_numpy()
print(cell_types)
# generate a link based on a hemilineage
# local and intersegmental preMNs

class_df = leg_df.loc['local']
for ct in cell_types:
    if type(ct) is not str and np.isnan(ct):
        continue
    seg_ids = class_df.loc[idx[:,:,:,ct],:].index.get_level_values('segID')
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = ct,
                seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('leg_jsons/{}.json'.format(ct), "w") as outfile:
        json.dump(state, outfile)

# seg_ids = class_df.loc[idx[:,:,:,ct],:].index.get_level_values('segID')
# formatted_ids = [str(x) for x in seg_ids]

# state = make_json(name = ct,
#             seg_ids = formatted_ids)
# new_id = client.state.upload_state_json(state)
# client.state.build_neuroglancer_url(new_id, ngl_base)

# # save json 
# with open('leg_jsons/{}.json'.format(ct), "w") as outfile:
#     json.dump(state, outfile)

ValueError: Length mismatch: Expected axis has 1460 elements, new values have 1488 elements

In [803]:
# if you want a link
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5986275473489920'